# NY Times API 

So far, we have had exposure to working with requests library in Python. Using our knwoledge of APIs, we are going to use the NY Times API and collect a set of data.Please note that the NY Times API will not return full text for available news article. However, it will provide us with useful metadata such as subject terms, abstract, and updated-date, urls to images, and any other information a developer might seek when scraping the site. 

The main steps for this tutotal are the following:
1. Fetch data from NY Times API
2. Load JSON data using requests
3. parse nested JSON for title of article and data updated
4. **Advanced**: Use a visualization library of your choice, visually represent the data. 

First step is to [sign up](https://developer.nytimes.com/signup) for getting an API key. Take a moment to look through the link. Notice that the available api-key requests are separated by the _type_ of request. This is helpful for us as developers but it also provides user meta data to NY Times. For example, client metadata from api-key requests can be used by NY Times for analysis ([BIA](https://en.wikipedia.org/wiki/Business_intelligence)), building future APIs, improving existing ones, or providing additional API wrappers. In an organizational setting, generated meta data from the client side is just as important as receiving data from the server. User data has helped NY Times improve its developer tools such as providing new APIs, corresponding wrappers, and improving data accessibility. 

In the sign-up page, fill out your information to receive your api-key. For the purposes of this tutorial, the Top Stories API key was requested and used. Feel free to pick any of the other available APIs. See a [list](https://developer.nytimes.com/) of available APIs to follow through this tutorial. Once you have received the api key in your e-mail, store it as a variable in your python script. 

One of the main reasons the NY Times was selected for the purposes of this course is to receive a visual understanding between dynamic vs. static URL paramters. , URL To understand how a request can be made (GET request) NY Times has made a [GUI interaface](https://developer.nytimes.com/top_stories_v2.json) available. 

For this notebook the following modules will be used:

In [29]:
import requests
import pandas as pd
import json
import sys

## 1. Fetching Data
After importing the required modules, retrieving the necessary API key, choose a parameter you want to work with. This can be a single parameter (e.g. politics) or a list of parameters you want to iterate through as you construct your url. For the purposes of simplicity, I have chosen a single parameter. 

In [4]:
#sign up and get a key -- I chose Top Stories
key = "a1b6d14c03494629990937518357c475"
# within Top Stories, there are list of parameters to work with. Choose one param or a list of params to iterate through. 
url = 'https://api.nytimes.com/svc/topstories/v2/politics.json?q=&api-key=%s' % key
print(url)

https://api.nytimes.com/svc/topstories/v2/politics.json?q=&api-key=a1b6d14c03494629990937518357c475


## 2. Load the Data as a JSON

Now that we have constructed our url, feel free to vist and view JSON object you will be requesting in the next section. A [GET](https://developer.mozilla.org/en-US/docs/Web/HTTP/Methods/GET) request is performed. Error handling is also put in place for this section to ensure the right object is returned. This prevents confusion later on in the code because the json object loaded and returned gets used in part 3 and 4. 

In [27]:
def data_fetch(a):
    try:
        r = requests.get(url, timeout=100)
        
        if r.status_code != 200:
            r = 'N/A'
            return r
        else:
            obj = r.json()
            return obj
        
    except requests.exceptions.RequestException as error:
        print(error)
        sys.exit(1)

main_obj = data_fetch(url)
print(main_obj)

{'status': 'OK', 'copyright': 'Copyright (c) 2018 The New York Times Company. All Rights Reserved.', 'section': 'politics', 'last_updated': '2018-01-18T20:18:03-05:00', 'num_results': 20, 'results': [{'section': 'U.S.', 'subsection': 'Politics', 'title': 'House Passes Short-Term Spending Bill, Setting Up Shutdown Battle in Senate', 'abstract': 'The successful vote in the House provided only a faint glimmer of hope that a crisis could be averted before much of the government exhausts its funds at midnight on Friday.', 'url': 'https://www.nytimes.com/2018/01/18/us/politics/government-shutdown-house-vote.html', 'byline': 'By THOMAS KAPLAN and SHERYL GAY STOLBERG', 'item_type': 'Article', 'updated_date': '2018-01-18T19:49:45-05:00', 'created_date': '2018-01-18T09:30:40-05:00', 'published_date': '2018-01-18T09:30:40-05:00', 'material_type_facet': '', 'kicker': '', 'des_facet': ['United States Politics and Government', 'Law and Legislation', 'Shutdowns (Institutional)', "State Children's Hea

Now that we have our JSON loaded, let's take a moment to understand the code above. For line 3, the get request is slightly different from what we have come across. The [timeout](http://docs.python-requests.org/en/master/user/quickstart/#timeouts) argument is used to instruct Requests on how long to wait on a response. For the example, 100 seconds has been designated but this can be smaller or bigger. This 100 second is not a limitation on how long the script has to load our needed data. Instead, this is an instruction to raise exception errors if 100 seconds elapsed but no byte of data has been received. 

We also have included a check to ensure the correct status_code in our request is getting returned. Different [status code](https://developer.mozilla.org/en-US/docs/Web/HTTP/Status) indicates different responses the server sends when a client requests data using [Hyper Transfer Text Protocol](https://developer.mozilla.org/en-US/docs/Web/HTTP). Since a 200 status code indicates that the request has been successful, we are checking for any event where the response comes back without the request being successful. 

Line 12 focuses on any ambiguous error that could have occured. Instead of having a statement for base class exception errors, this statement can be modified to specifically look for ConnectionErrors. In order to write more robust code with better error handling in the future, take a look at the [Exceptions doc](http://docs.python-requests.org/en/master/api/#exceptions). 

## 3. Parsing Nested JSON

A JSON object is contained withing {}. Inside the JSON object, there is always a string key paired with a value that can be a string, number, another object, array, boolean, or null. A nested JSON object is when a key of a JSON contains another JSON object as its value. It is practical to understand the construction of large JSON objects by targetting the value of a key within a nested JSON object. For the purposes of this tutorial, we are going to look for the titles of all our articles under the Politics category in Top Stories the API has retured. 

First, let's examine the keys that appear in the JSON object. 

In [31]:
def parse_json(obj):
    data = obj.get('results')
    i = len(data)

    #Accessing the Title, published_date, and abstracts of each article
    title = []
    date = []
    ab = []
    c = 0
    for item in data[c]:
        val = data[c].get('title')
        title.append(val)
        
        dt = data[c].get('published_date')
        date.append(dt)
    
        a = data[c].get('abstract')
        ab.append(a)
        
        c += 1
        
    #creating a pandas data frame
    df = pd.DataFrame({'Published Date': date, 'Article Title': title, 'Abstract': ab})
    print(df)
    return df
        
print(parse_json(main_obj))


In this step, we have sifted through a nested JSON object, extracted the specific elements needed, and reformatted the required information into a Pandas Data Frame. Pandas data frames are crucial for working with data sets in Python. The data from this format can now be used for further textual analysis and data visualization. 

## 4. Visualization

The purposes of data visualization is usually based on a case by case basis. The purposes of good data visualization is to easily communicate your findings to someone else. Visualization of data needs to be functional, clean, and easy to read. Graphs and charts should follow good design principles that convey the purpose of your project to a new audience. Data visualization is a fast growing and vast field. For the purposes of this tutorial, this is an optional step. 


                                             Abstract  \
0   The successful vote in the House provided only...   
1   In an early morning tweet, President Trump con...   
2   The 2020 race for the White House is already u...   
3   John F. Kelly told Hispanic members of Congres...   
4   A review of President Trump's statements on Tw...   
5   The plans suggest the administration no longer...   
6   A chief architect of the 1965 march on Washing...   
7   The decision makes it likely the 2018 election...   
8   The Fed is likely to get a new chairman next m...   
9   Jerry Lee, the former officer, had repeated co...   
10  Yukking it up with Ann Coulter and Janice Min ...   
11  The Senate sent a bill to President Trump exte...   
12  Kenneth Marcus’s potential return to the Educa...   
13  The Trump administration announced a new “cons...   
14  California’s senators asked Jeff Sessions why ...   
15  A Gallup survey of people in 134 countries fou...   
16  Kentucky’s new Medicaid wai

KeyError: 0